# 1章 セレクションバイアスとRCT

## 1.4 Rによるメールマーケティングの効果の検証
- pythonでやる

### 1.4.1

In [11]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

In [3]:
male_df = email_data.copy()[email_data["segment"]!="Womens E-Mail"]
male_df["treatment"] = (male_df["segment"]=="Mens E-Mail").astype(int)

In [6]:
male_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0


### 1.4.2

In [4]:
summary_by_segment = male_df.groupby("treatment").agg(
    conversion_rate=pd.NamedAgg("conversion", "mean"),
    spend_mean=pd.NamedAgg("spend","mean"),
    count=pd.NamedAgg("spend","count")
)

In [5]:
summary_by_segment

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


In [9]:
rct_ttest = stats.ttest_ind(
    male_df[male_df["treatment"]==1]["spend"],
    male_df[male_df["treatment"]==0]["spend"]
)

In [10]:
rct_ttest

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

### 1.4.3

In [42]:
seed = 1

In [43]:
obs_rate_c = 0.5
obs_rate_t = 0.5
bias = (male_df.history>300) | (male_df.recency<6) | (male_df.channel=="Multichannel")
biased_data = pd.concat([
    male_df[(male_df["treatment"]==0) & bias].sample(frac=obs_rate_c, random_state=seed),
    male_df[(male_df["treatment"]==0) & ~bias],
    male_df[(male_df["treatment"]==1) & ~bias].sample(frac=obs_rate_t, random_state=seed),
    male_df[(male_df["treatment"]==1) & bias]
], ignore_index=True)

In [44]:
biased_data.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,8,5) $500 - $750,572.65,1,0,Urban,1,Web,No E-Mail,0,0,0.0,0
1,5,1) $0 - $100,42.38,1,0,Urban,1,Phone,No E-Mail,1,0,0.0,0
2,1,"7) $1,000 +",3003.48,1,1,Urban,1,Phone,No E-Mail,0,0,0.0,0
3,1,5) $500 - $750,662.10,0,1,Urban,1,Web,No E-Mail,0,0,0.0,0
4,5,1) $0 - $100,44.37,0,1,Urban,0,Web,No E-Mail,0,0,0.0,0


In [45]:
summary_by_segment_biased = biased_data.groupby('treatment').agg(
    conversion_rate=pd.NamedAgg("conversion", "mean"),
    spend_mean=pd.NamedAgg("spend","mean"),
    count=pd.NamedAgg("spend","count")
)

In [46]:
summary_by_segment_biased

,conversion_rate,spend_mean,count
treatment,,,
0,0.004540,0.557954,14757
1,0.013572,1.541704,17168


In [47]:
rct_ttest_biased = stats.ttest_ind(
    biased_data[biased_data["treatment"]==1]["spend"],
    biased_data[biased_data["treatment"]==0]["spend"]
)

In [48]:
rct_ttest_biased

Ttest_indResult(statistic=5.595867225527973, pvalue=2.2131984133654547e-08)

In [51]:
# biasのせいでp値が小さくなっている
rct_ttest_biased.pvalue < rct_ttest.pvalue

True